In [1]:
import os
import numpy as np
import shutil
import matplotlib
import matplotlib.pyplot as plt
from astropy import table
from astropy.io import ascii
import cmasher as cmr
import dynamite as dyn
from plotbin import display_pixels

In [ ]:
# Get the DYNAMITE models
fname = 'F2_config11_with_pm.yaml'
c = dyn.config_reader.Configuration(fname,
                                    reset_logging=True,
                                    user_logfile='test_nnls',
                                    reset_existing_output=True)
stars = c.system.get_unique_triaxial_visible_component()
# print(f'stars.kinematic_data[0]:\n{stars.kinematic_data[0]}\n\nstars.kinematic_data[1]:\n{stars.kinematic_data[1]}')
_ = dyn.model_iterator.ModelIterator(config=c)

[INFO] 19:48:57 - dynamite.config_reader.Configuration - Config file F2_config11_with_pm.yaml read.
[INFO] 19:48:57 - dynamite.config_reader.Configuration - io_settings...
[INFO] 19:48:57 - dynamite.config_reader.Configuration - Output directory tree F2_output11/ removed.
[INFO] 19:48:57 - dynamite.config_reader.Configuration - Output directory tree: F2_output11/.
[INFO] 19:48:57 - dynamite.config_reader.Configuration - system_attributes...
[INFO] 19:48:57 - dynamite.config_reader.Configuration - model_components...
[INFO] 19:48:57 - dynamite.physical_system.Component - bh: DYNAMITE will currently ignore the mandatory attribute contributes_to_potential.
[INFO] 19:48:57 - dynamite.config_reader.Configuration - dh... ignored
[INFO] 19:48:57 - dynamite.physical_system.Component - stars: DYNAMITE will currently ignore the mandatory attribute contributes_to_potential.
[INFO] 19:48:57 - dynamite.kinematics.Kinematics - propermotions: vx_sigma=158.3257256428834, dvx=30.975609756097583=0.19564

In [ ]:
p = dyn.plotter.Plotter(c)

In [ ]:
# Structure of proper motion input data
for i, k in enumerate(stars.kinematic_data):
    if isinstance(k, dyn.kinematics.ProperMotions):
        i_pm = i
        pm_data = k.data
        pm_input = k
        break
print('Shapes of proper motion input data:')
for k in sorted(pm_data):
    print(f"{k}: {pm_data[k].shape}")
print(f"{pm_data['vxrange'] = }, {pm_data['vyrange'] = }")
h2d = pm_input.as_histogram2d()
print(f"{pm_input.hist_width = }, {pm_input.hist_bins = }")
print(f"2d histogram velocity edges  {len(h2d.xedg[0]) = }, vx-min/max: {h2d.xedg[0][0]}, {h2d.xedg[0][-1]}")
print(f"2d histogram vel bin centers {len(h2d.x[0])    = }, vx-min/max: {h2d.x[0][0]}, {h2d.x[0][-1]}")
print(f"2d histogram dvx:                               {h2d.dx[0][0]}")

In [ ]:
# Strange: nstarbin > sum of all PM_2dhist entries almost everywhere
for i in range(0, pm_data['PM_2dhist'].shape[0], 4):  # don't print everything...
    print(f"Spatial bin {i}:\t{pm_data['nstarbin'][i]=},\t{np.sum(pm_data['PM_2dhist'][i])=}")
print(f"{np.argmin(pm_data['nstarbin'] - np.sum(pm_data['PM_2dhist'], axis=(1,2))) = }")
print(f"{np.argmax(pm_data['nstarbin'] - np.sum(pm_data['PM_2dhist'], axis=(1,2))) = }")

In [ ]:
# Plot the input data
h2d = pm_input.as_histogram2d()
# First, a few individual spatial bins...
for sp_bin_idx in(11, 48, 75):
    print(f'{sp_bin_idx=}')
    _ = p.hist2d_plot(h2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)
# Now the global 2d histogram...
h2d_global = dyn.kinematics.Histogram2D(xedg=h2d.xedg,
                                        y=np.sum(h2d.y, axis=3)[:,:,:,np.newaxis],
                                        normalise=False)
_ = p.hist2d_plot(h2d_global, orb_idx=0, sp_bin_idx=0, show_1d=True, empty_bins=True)

In [ ]:
# Experimental, does NOT work yet: quiver plot -> problem in get_mean or plotting...?
v_mean = pm_input.as_histogram2d().get_mean()
x, y = pm_data['xbin'], pm_data['ybin']
u, v = v_mean[0][0], v_mean[1][0]  # first index: vx/vy, second index: orb_idx
plt.quiver(x, y, u, v, color='g')
plt.axis('equal')

In [ ]:
# Get best model
best_model_idx = c.all_models.get_best_n_models_idx(n=1)[0]
model = c.all_models.get_model_from_row(best_model_idx)
orblib = model.get_orblib()
print(f'{orblib.parset=}')
_ = model.get_weights(orblib)
orb_weights = model.weights
hist2d_orblib = orblib.vel_histograms[i_pm]
# print(f'{hist2d_orblib.y.shape = }')

In [ ]:
# plot pm histograms for all spatial bins
n_bins = hist2d_orblib.y.shape[-1]

vx_range = [hist2d_orblib.x[0].min(), hist2d_orblib.x[0].max()]
vy_range = [hist2d_orblib.x[1].min(), hist2d_orblib.x[1].max()]
ratio = (vy_range[1]-vy_range[0]) / (vx_range[1]-vx_range[0])
print('Ranges are different from input data due to velocity scaling => FIXME: add analogue to rebin_orblib_to_observations() to ProperMotions.transform_orblib_to_observables()')
print(f'{vx_range=}, {vy_range=}')
print(f"vx-range from xedg: {hist2d_orblib.xedg[0][0]}, {hist2d_orblib.xedg[0][-1]}")
print(f"vy-range from xedg: {hist2d_orblib.xedg[1][0]}, {hist2d_orblib.xedg[1][-1]}")

data = np.einsum('ijkl,i', hist2d_orblib.y, orb_weights)
# data = np.dot(hist2d_orblib.y.T, orb_weights).T  # specific orbit: hist2d.y[orb_idx,:,:,bin_idx]

fig = plt.figure(figsize=(20, 20 * (n_bins // 4 // 4 + 2) * ratio))
for bin_idx in range(0, n_bins, 1):  ##################################
    ax = plt.subplot(n_bins // 4 + (1 if n_bins % 4 > 0 else 0), 4, bin_idx + 1)
    # im = ax.imshow(data, aspect='equal', interpolation='bilinear', #cmap=cm.RdYlGn,
    im = ax.imshow(data[:,:,bin_idx], aspect='equal', interpolation='none', #cmap=cm.RdYlGn,
                   origin='lower', extent=vx_range + vy_range,
                   vmax=data.max(), vmin=0)
    fig.colorbar(im, ax=ax, shrink=1)
    ax.set_title(f'Bin {bin_idx}')
plt.show()

In [ ]:
# plot pm histograms for specific bin
n_orbits = hist2d_orblib.y.shape[0]

bin_idx = 31

vx_range = [hist2d_orblib.x[0].min(), hist2d_orblib.x[0].max()]
vy_range = [hist2d_orblib.x[1].min(), hist2d_orblib.x[1].max()]
ratio = (vy_range[1]-vy_range[0]) / (vx_range[1]-vx_range[0])
print(f'{vx_range=}, {vy_range=}')

fig = plt.figure(figsize=(20, 20 * (n_orbits // 4 // 4 + 2) * ratio))
for orb_idx in range(0, n_orbits, 1):  ##################################
    data = hist2d_orblib.y[orb_idx,:,:,bin_idx]
    ax = plt.subplot(n_orbits // 4 + (1 if n_orbits % 4 > 0 else 0), 4, orb_idx + 1)
    im = ax.imshow(data, aspect='equal', interpolation='none', #cmap=cm.RdYlGn,
                   origin='lower', extent=vx_range + vy_range,
                   vmax=data.max(), vmin=0)
    fig.colorbar(im, ax=ax)
    ax.set_title(f'Bin {bin_idx}, orbit {orb_idx}')

plt.show()

In [ ]:
# plot the pm histograms' mean values (doesn't fit at all yet...)
# mostly from plotter.py

def create_plot(dp_args, data):
    # get aperture and bin data
    x = dp_args['x']
    y = dp_args['y']
    dx = dp_args['dx']
    grid = dp_args['idx_bin_to_pix']
    angle_deg = dp_args['angle']
    # Only select the pixels that have a bin associated with them.
    s = np.ravel(np.where((grid >= 0)))

    #fhist, _ = np.histogram(grid[s], bins=len(data))
    #data = data / fhist
    # plot settings
    #data_min = min(data[grid[s]] / max(data))
    #data_max = max(data[grid[s]] / max(data))
    # The galaxy has NOT already rotated with PA to align major axis with x
    map1 = cmr.get_sub_cmap('twilight_shifted', 0.05, 0.6)
    kw_display_pixels1 = dict(pixelsize=dx,
                              angle=angle_deg,
                              colorbar=True,
                              nticks=7,
                              # cmap='sauron')
                              cmap=map1)
    # PLOT THE DATA
    #plt.figure()
    #c_c = data[grid[s]] / max(data)
    c_c = data[grid[s]]
    display_pixels.display_pixels(x, y, c_c,
                                  vmin=min(data[grid[s]]), vmax=max(data[grid[s]]),
                                  label='velocity',
                                  **kw_display_pixels1)
    #plt.gca().set_title(title)
    #plt.show()

fig = plt.figure(figsize=(15, 5))

n_rows, n_cols = 2, 2

mean_vel = pm_input.as_histogram2d().get_mean()  # tuple (mean_velx(n_orbits, n_apertures), mean_vely(n_orbits, n_apertures))

ax = plt.subplot(n_rows, n_cols, 1)
ax.set_title(f'Data pm x')
mean = mean_vel[0][0]
create_plot(pm_input.dp_args, mean)

ax = plt.subplot(n_rows, n_cols, 2)
ax.set_title(f'Data pm y')
mean = mean_vel[1][0]
create_plot(pm_input.dp_args, mean)

data = np.einsum('ijkl,i', hist2d_orblib.y, orb_weights)[np.newaxis]
hist_plot = dyn.kinematics.Histogram2D(xedg=hist2d_orblib.xedg,
                                       y=data,
                                       normalise=False)
mean_vel = hist_plot.get_mean()  # tuple (mean_velx(n_orbits, n_apertures), mean_vely(n_orbits, n_apertures))

ax = plt.subplot(n_rows, n_cols, 3)
ax.set_title(f'Model pm x')
mean = mean_vel[0][0]
create_plot(pm_input.dp_args, mean)

ax = plt.subplot(n_rows, n_cols, 4)
ax.set_title(f'Model pm y')
mean = mean_vel[1][0]
create_plot(pm_input.dp_args, mean)

In [ ]:
# Orblib pm vs input pm for specific sp_bin_idx
sp_bin_idx_list = [25, 48, 70]

data = np.dot(hist2d_orblib.y.T, orb_weights).T
print(f'{data.shape = }')
print(f'{np.max(hist2d_orblib.y)=}')
for sp_bin_idx in sp_bin_idx_list:
    print(f'{sp_bin_idx = }')
    hist_2d = dyn.kinematics.Histogram2D(xedg=hist2d_orblib.xedg,
                                         y=data[np.newaxis,:,:,:],
                                         normalise=False)
    _ = p.hist2d_plot(hist_2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)
    _ = p.hist2d_plot(pm_input.as_histogram2d(), orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)

In [ ]:
import datetime
datetime.datetime.now()

In [ ]:
f=np.load('Francisco/rot_mod_d5kpc_i00deg/kinematics_hist2d.npz')

In [ ]:
f['PSF_weight'], f['PSF_weight']